In [11]:
import pandas as pd

In [12]:
df_co2 = pd.read_csv('./lstm_datasets/co2_prediction.csv')
df_co2.head()

df_energy = pd.read_csv('./lstm_datasets/energy_prediction.csv')
df_energy.head()

df_literacy = pd.read_csv('./lstm_datasets/literacy_prediction.csv')
df_literacy.head()

df_expected_schooling = pd.read_csv('./lstm_datasets/expected_schooling_prediction.csv')
df_expected_schooling.head()

df_co2_2 = pd.read_csv('./lstm_datasets/co2#2_prediction.csv')
df_co2_2.head()

df_gdp = pd.read_csv('./lstm_datasets/gdp_prediction.csv')
df_gdp.head()

df_internet = pd.read_csv('./lstm_datasets/internet_prediction.csv')
df_internet.head()

df_gini = pd.read_csv('./lstm_datasets/gini_prediction.csv')
df_gini.head()

df_mean_schooling = pd.read_csv('./lstm_datasets/mean_schooling_prediction.csv')
df_mean_schooling.head()


,Country,2024 Mean Schooling,2025 Mean Schooling,2026 Mean Schooling,2027 Mean Schooling,2028 Mean Schooling
0,Afghanistan,2.991220,3.075291,3.151216,3.218589,3.276677
1,Angola,5.709934,5.811828,5.931969,6.051953,6.165771
2,Albania,11.611897,11.720772,11.818372,11.904657,12.009837
3,Andorra,10.526522,10.524909,10.523480,10.522296,10.521358
4,United Arab Emirates,13.743556,14.019395,14.254596,14.528773,14.895000


In [13]:
weighting = {
    'CO2': -0.0339,
    'CO2#2': 0.1111,
    'Energy': -0.1315,
    'GDP': 0.0203,
    'Gini': -0.3958,
    'Internet': 0.6896,
    'Literacy': 0.9832,
    'Expected Schooling': 0.0757,
    'Mean Schooling': 0.0202
}

In [14]:
# Create a new DataFrame for CDI predictions
cdi_predictions = pd.DataFrame()
cdi_predictions['Country'] = df_co2['Country']  # Use any of the prediction DataFrames for countries

# Calculate CDI for each year
for year in range(2024, 2029):
    # Get predictions for each variable for the current year
    values = {
        'CO2': df_co2[f'{year} CO2'] * weighting['CO2'],
        'CO2#2': df_co2_2[f'{year} CO2#2'] * weighting['CO2#2'],
        'Energy': df_energy[f'{year} Energy'] * weighting['Energy'],
        'GDP': df_gdp[f'{year} GDP'] * weighting['GDP'],
        'Gini': df_gini[f'{year} Gini'] * weighting['Gini'],
        'Internet': df_internet[f'{year} Internet'] * weighting['Internet'],
        'Literacy': df_literacy[f'{year} Literacy'] * weighting['Literacy'],
        'Expected Schooling': df_expected_schooling[f'{year} Expected Schooling'] * weighting['Expected Schooling'],
        'Mean Schooling': df_mean_schooling[f'{year} Mean Schooling'] * weighting['Mean Schooling']
    }
    
    # Calculate CDI for the current year
    cdi_year = sum(values.values())
    cdi_predictions[f'{year} CDI'] = cdi_year

# Export to CSV
cdi_predictions.to_csv('lstm_datasets/cdi_predictions.csv', index=False)
print("\nCDI predictions exported to 'lstm_datasets/cdi_predictions.csv'")

# Display first few rows
print("\nFirst few rows of CDI predictions:")
print(cdi_predictions.head())

# Plot CDI predictions for selected countries
selected_countries = ['United States', 'China', 'Japan', 'Germany', 'United Kingdom']

plt.figure(figsize=(12, 6))
for country in selected_countries:
    if country in cdi_predictions['Country'].values:
        values = cdi_predictions[cdi_predictions['Country'] == country].iloc[0, 1:].values
        plt.plot(range(2024, 2029), values, label=country)

plt.title('CDI Predictions (2024-2028)')
plt.xlabel('Year')
plt.ylabel('CDI')
plt.legend()
plt.grid(True)
plt.show()


CDI predictions exported to 'lstm_datasets/cdi_predictions.csv'

First few rows of CDI predictions:
        Country    2024 CDI    2025 CDI    2026 CDI    2027 CDI    2028 CDI
0   Afghanistan  124.203377  133.375151  138.537187  139.522017  140.696928
1        Africa  257.025937  260.298651  263.913455  265.913990  267.150974
2  Africa (GCP)  229.784017  232.039660  234.213150  235.944340  237.309424
3       Albania  377.050560  381.389932  391.379534  402.914392  412.280117
4       Algeria  937.005422  927.758085  908.135542  892.899693  885.792951


NameError: name 'plt' is not defined